In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import numpy as np
import random
import time
from urllib.parse import urlparse
from scipy import sparse
import math

In [2]:
import elasticsearch
import elasticsearch_dsl

In [3]:
es = Elasticsearch()

In [4]:
crawled = {}
results = elasticsearch.helpers.scan(es,
    index="crawled_data_merged",
    doc_type="doc",
    preserve_order=True,
    query={"query": {"match_all": {}}},
)



In [5]:
for item in results:
    crawled[item['_id']] = item['_source']

In [6]:
i = 0
graph = {}

for link in crawled:
    if link not in graph:
        graph[link] = {
            "inlinks" : [],
            "outlinks": []
        }

    outlink = []
    
    for ol in crawled[link]["OUTLINK"]:
        if ol in crawled:
            outlink.append(ol)
        else:
            o = urlparse(ol)
            outlink.append(o.scheme + "://" + o.netloc)
    graph[link]["outlinks"] = list(set(outlink))
    
    
    for in_l in graph[link]["outlinks"]:
        if in_l not in graph:
            graph[in_l] = {
                "inlinks" : [],
                "outlinks": []
            }           
        graph[in_l]["inlinks"].append(link)

    if i % 1000 == 0:
        print(i,end =" ")
    i = i + 1
                

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 

In [7]:
for node in graph:
    graph[node]["inlinks"] = list(set(graph[node]["inlinks"]))
    graph[node]["outlinks"] = list(set(graph[node]["outlinks"]))

In [8]:
ittr = 100

In [9]:
def HITS(term, ittr, eps):
#     root_set = []
#     base_set = []

    
    body = {
        "query": {
                "match": {
                  "TEXT": term        
                }
            },
        "size" : 1000
    }

    terms = es.search(index = "crawled_data_merged",doc_type="doc", body=body)['hits']['hits']

    root_set = {}
    base_set = {}

    
    for link in terms:
        root_set[link['_id']] = True
        base_set[link['_id']] = True
        
    itr = 0
    
    
    while(len(set(root_set)) <= 10000):
        for link in root_set:
#             if link.find("quotes.liberty-tree.ca") != -1:
#                 continue

            _set = []
            _set = _set + graph[link]["inlinks"]
            _set = _set + graph[link]["outlinks"]
            _set = list(set(_set))
            
            if len(_set) > 200:
                _set = np.array(_set)
                _set = _set[random.sample(range(0, len(_set)), 200)]
                _set = _set.tolist()
            
            for _url in _set:
                base_set[_url] = True
            
            if(len(base_set) > 10000 and itr != 0) :
                break

        for link in base_set:
#             if link.find("quotes.liberty-tree.ca") != -1:
#                 continue
            root_set[link] = True
        itr += 1
        
        if(itr == 5):
            break
        print(itr, end=", ")

        
    base_set_mapping = {}
    base_rev_set_mapping = {}

    id_bs = 0
    for url in root_set:
        base_set_mapping[url] = id_bs
        base_rev_set_mapping[id_bs] = url
        id_bs += 1
        
    adjcency = np.zeros((len(base_set), len(base_set)))
    
    st = time.time()
    for link in base_set:
        _id = base_set_mapping[link]

        for out in graph[link]['outlinks']:

            if out not in base_set_mapping:
                continue

            _out_id = base_set_mapping[out]
            if _id == _out_id:
                continue
            adjcency[_id][_out_id] = 1

        for ins in graph[link]['inlinks']:

            if ins not in base_set_mapping:
                continue


            _in_id = base_set_mapping[ins]
            if _id == _in_id:
                continue
            adjcency[_out_id][_id] = 1
    
    print(time.time() - st)
        
    hubs = np.ones(len(base_set))
    auth = np.ones(len(base_set)) #transpose matrix
    
    auth = auth / math.sqrt(sum(np.square(auth)))
    hubs = hubs / math.sqrt(sum(np.square(hubs)))
    
    for i in range(ittr):
    
        auth_new = adjcency.T.dot(hubs)
        hubs_new = adjcency.dot(auth)

        auth_new = auth_new / math.sqrt(sum(np.square(auth_new)))
        hubs_new = hubs_new / math.sqrt(sum(np.square(hubs_new)))

        authdiff = np.abs(auth_new - auth)
        hubdiff = np.abs(hubs_new - hubs)

        if (authdiff > eps).any():
            auth = auth_new
            hubs = hubs_new
            continue
        else:    
            print (i, "found")
            break    
    
    with open(term.replace(" ", "_") +"_authority","w") as file:
        for id in auth.argsort()[::-1][0:500]:
            file.write(base_rev_set_mapping[id]+"    "+ str(auth[id]) +"\n")
            
    with open(term.replace(" ", "_") +"_hub","w") as file:
        for id in hubs.argsort()[::-1][0:500]:
            file.write(base_rev_set_mapping[id]+"    "+ str(hubs[id])+"\n")    

    return root_set, base_set_mapping, base_rev_set_mapping

In [21]:
base_set, base_set_mapping, base_rev_set_mapping = HITS("american revolutionary war", 1000, eps = 1.0e-6)

1, 2, 0.4417448043823242
38 found


In [11]:
# len(base_set)

In [12]:
# adjcency = np.zeros((len(base_set), len(base_set)))

In [13]:
# st = time.time()
# for link in base_set:
#     _id = base_set_mapping[link]
    
#     for out in graph[link]['outlinks']:
        
#         if out not in base_set_mapping:
#             continue

#         _out_id = base_set_mapping[out]
#         if _id == _out_id:
#             continue
#         adjcency[_id][_out_id] = 1
        
#     for ins in graph[link]['inlinks']:
        
#         if ins not in base_set_mapping:
#             continue

        
#         _in_id = base_set_mapping[ins]
#         if _id == _in_id:
#             continue
#         adjcency[_out_id][_id] = 1

    
    
        
# print(time.time() - st)
        

In [14]:
# max(sum(adjcency))

In [15]:
# hubs = np.ones(len(base_set))
# auth = np.ones(len(base_set)) #transpose matrix

In [16]:
# auth = auth / math.sqrt(sum(np.square(auth)))
# hubs = hubs / math.sqrt(sum(np.square(hubs)))

In [17]:
# eps = 1.0e-6

In [18]:
# for i in range(1000):
    
#     auth_new = adjcency.T.dot(hubs)
#     hubs_new = adjcency.dot(auth)
    
#     auth_new = auth_new / math.sqrt(sum(np.square(auth_new)))
#     hubs_new = hubs_new / math.sqrt(sum(np.square(hubs_new)))

#     authdiff = np.abs(auth_new - auth)
#     hubdiff = np.abs(hubs_new - hubs)
    
#     print(i , end = ", ")
#     if (authdiff > eps).any():
#         auth = auth_new
#         hubs = hubs_new
#         continue
#     else:    
#         print ("found")
#         break    

In [19]:
# for id in auth.argsort()[::-1][0:500]:
#     print(base_rev_set_mapping[id])

In [20]:
# for id in hubs.argsort()[::-1][0:500]:
#     print(base_rev_set_mapping[id])